In [1]:
import os
import subprocess
import sys
from typing import List
from pyspark.sql import SparkSession
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the base directory
jars_home = '/workspace/delta-jars'

# Required core JARs
jars_list = [
    # Delta Lake
    f"{jars_home}/delta-spark_2.12-3.2.1.jar",
    f"{jars_home}/delta-storage-3.2.1.jar",
    # AWS
    f"{jars_home}/hadoop-aws-3.3.2.jar",
    f"{jars_home}/aws-java-sdk-bundle-1.12.261.jar",
    # Kyuubi
    f"{jars_home}/kyuubi/externals/engines/spark/kyuubi-spark-sql-engine_2.12-1.9.0.jar",
    f"{jars_home}/kyuubi/externals/engines/spark/kyuubi-common_2.12-1.9.0.jar"
]

# Convert to comma-separated string
jars = ",".join(jars_list)

# Create SparkSession using the builder pattern
builder = (SparkSession.builder
           .appName("DeltaExample")
           .master("local[*]")
           # Add debug configurations
           .config("spark.hadoop.fs.s3a.connection.maximum", "1")
           .config("spark.hadoop.fs.s3a.attempts.maximum", "1")
           .config("spark.hadoop.fs.s3a.connection.timeout", "5000")
           .config("spark.hadoop.fs.s3a.impl.disable.cache", "true")
           .config("spark.hadoop.fs.s3a.debug.detailed.exceptions", "true")
           # Add jars directly
           .config("spark.jars", ",".join(jars))
           .config("spark.driver.extraClassPath", ",".join(jars))
           .config("spark.executor.extraClassPath", ",".join(jars))
           # Delta Lake configurations
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
           # S3/MinIO configurations
           .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
           .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
           .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
           .config("spark.hadoop.fs.s3a.path.style.access", "true")
           .config("fs.s3a.metrics.enabled", "false")
           .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
           .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
           .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
           # Additional Delta Lake configurations
           .config("spark.delta.logStore.class", "io.delta.storage.S3SingleDriverLogStore")
           .config("spark.hadoop.fs.s3a.fast.upload", "true")
           .config("spark.hadoop.fs.s3a.multipart.size", "104857600")
           .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse"))

# Stop any existing session
if 'spark' in locals():
    spark.stop()


# Create the session
spark = builder.enableHiveSupport().getOrCreate()

# Initialize Delta Lake settings
# spark.sql("SET spark.databricks.delta.formatCheck.enabled=false")

# Access the SparkContext
# sc = spark.sparkContext

# Set the log level to INFO
# sc.setLogLevel("DEBUG")

# Test DataFrame
data = [(1, "John"), (2, "Jane")]
df = spark.createDataFrame(data, ["id", "name"])
df.show()

# First verify the S3 connection by listing the bucket
try:
    # Try to write to a simple parquet file first to test S3 connection
    print("Testing S3 connection with parquet write...")
    df.write.format("parquet").mode("overwrite").save("s3a://wba/test.parquet")
    print("S3 connection successful")

    print("Attempting to write Delta table...")
    df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .option("delta.compatibility.symlinkFormatManifest.enabled", "false") \
        .save("s3a://wba/example-table")
    print("Successfully wrote Delta table")

except Exception as e:
    print(f"Error: {str(e)}")
    print("\nTrying local filesystem instead...")
    try:
        local_path = "/tmp/test-delta-table"
        df.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .save(local_path)
        print(f"Successfully wrote to {local_path}")
    except Exception as local_e:
        print(f"Error writing to local filesystem: {str(local_e)}")

25/02/21 02:50:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/21 02:50:54 WARN DependencyUtils: Local jar /workspace/accelerator/materials/1-getting-started/notebooks/w does not exist, skipping.
25/02/21 02:50:54 WARN DependencyUtils: Local jar /workspace/accelerator/materials/1-getting-started/notebooks/o does not exist, skipping.
25/02/21 02:50:54 WARN DependencyUtils: Local jar /workspace/accelerator/materials/1-getting-started/notebooks/r does not exist, skipping.
25/02/21 02:50:54 WARN DependencyUtils: Local jar /workspace/accelerator/materials/1-getting-started/notebooks/k does not exist, skipping.
25/02/21 02:50:54 WARN DependencyUtils: Local jar /workspace/accelerator/materials/1-getting-started/notebooks/s does not exist, skipping.
25/02/21 02:50:54 WARN DependencyUtils: Local jar /workspace/accelerator/materials/1-getting-started/notebooks/p does not exist, skipping.
25/02/21 02:50:54 WARN 

+---+----+
| id|name|
+---+----+
|  1|John|
|  2|Jane|
+---+----+

Testing S3 connection with parquet write...


25/02/21 02:51:01 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/02/21 02:51:01 INFO MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
25/02/21 02:51:01 INFO MetricsSystemImpl: s3a-file-system metrics system started
25/02/21 02:51:03 INFO ParquetUtils: Using default output committer for Parquet: org.apache.parquet.hadoop.ParquetOutputCommitter
25/02/21 02:51:03 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
25/02/21 02:51:03 INFO FileOutputCommitter: FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
25/02/21 02:51:03 INFO SQLHadoopMapReduceCommitProtocol: Using user defined output committer class org.apache.parquet.hadoop.ParquetOutputCommitter
25/02/21 02:51:03 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
25/02/21 02:51:03 INFO FileOutputCommitter: FileOutputCommitter skip cl

S3 connection successful
Attempting to write Delta table...


25/02/21 02:51:05 INFO DeltaLog: Loading version 5.
25/02/21 02:51:06 INFO DeltaLogFileIndex: Created DeltaLogFileIndex(JSON, numFilesInSegment: 6, totalFileSize: 8579)
25/02/21 02:51:07 INFO FileSourceStrategy: Pushed Filters: 
25/02/21 02:51:07 INFO FileSourceStrategy: Post-Scan Filters: 
25/02/21 02:51:07 INFO CodeGenerator: Code generated in 227.857362 ms
25/02/21 02:51:07 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 235.2 KiB, free 433.8 MiB)
25/02/21 02:51:07 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 37.6 KiB, free 433.8 MiB)
25/02/21 02:51:07 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on f85d7ed12f17:36639 (size: 37.6 KiB, free: 434.3 MiB)
25/02/21 02:51:07 INFO SparkContext: Created broadcast 4 from toString at String.java:4220
25/02/21 02:51:07 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
25/02/21 02

Successfully wrote Delta table


25/02/21 02:51:20 INFO OptimisticTransaction: [tableId=95668314,txnId=5b257ce9] Committed delta #6 to s3a://wba/example-table/_delta_log
